## Chapter 10: Sequence Hacking, Hashing, and Slicing

In [ ]:
from array import array
import reprlib
import numbers
import math
import functools
import operator

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'    
    
    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self) -> str:
        # This lib gives us a limited-length representation of the array
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    # With the next two functions defined, objs of this class will be iterators
    def __len__(self):
        return len(self._components)
    
    # Now, when we grab a slice from a vector, we will get a similar vector, with just the sliced values.
    def __getitem__(self, index):
        cls = type(self)
        
        # If the index is actually a slice, then return a vector of equivalent type containing the sliced values.
        if isinstance(index, slice):
            return cls(self._components[index])
        # If it's just an int, return the value at that index
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = f"{cls.__name__} indices must be integers"
            raise TypeError(msg)
        
    # When extracting an obj attribute, the compiler first searches to see if any exist. If not, it calls
    # the following dunder method. Now, we use any of the "short-cut" names to determine a given value.
    # Eg with "v = Vector([1, 2, 3])", saying "v.y" corresponds to 2 because of the following declaration
    def __getattr__(self, name: str):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
            
        msg = f"{cls.__name__} has no attribute {name}"
        raise AttributeError(msg)
    
    # __getattr__ is read only. We must also construct a __setattr__ if we'd like to set values or control how the read-only parameters are read.
    # The function above has a problem that if someone says "v.x = 4", this will create a new parameter called x within v. Therefore, we must throw
    # an error upon setting any of the shortcut names.
    def __setattr__(self, name: str, value) -> None:
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = f'read-only attribute {name}'
            elif name.islower():
                error = f"can't set attributes 'a' to 'z' in {cls.__name__}"
            else:
                error = ""
            
            if error:
                raise AttributeError(error)
            
        super().__setattr__(name, value)
        
    def __eq__(self, o: object) -> bool:
        # return tuple(self) == tuple(o)  ->  This is highly inefficient for large vectors since we are building a new tuple from scratch and
        #                                     must be build by copy. Also, any data struct that happens to have the same numbers
        #                                     will still come out equal. Therefore, we do the following instead...
        
        if (type(self) != type(o)) or (len(self) != len(o)):
            return False
        
        # The following is great and pythonic but still not ideal cause for loops in python are slow.
        # for a, b in zip(self, o):
        #     if a != b:
        #         return False
        # return True
        
        return all(a == b for a, b in zip(self, o)) # Best solution!
        
    def __hash__(self) -> int:
        # hashes = (hash(x) for x in self._components)  ->  Also works but preferred method is below
        hashes = map(hash, self._components)
        
        # The reduce func uses an operator to aggregate an iterator into a single value. The "0" is the initial value
        return functools.reduce(operator.xor, hashes, 0)

In [3]:
from itertools import chain # Basically flattens iterables then horizontally concatenates them 
list(chain('hello', ' ', 'friend'))


['h', 'e', 'l', 'l', 'o', ' ', 'f', 'r', 'i', 'e', 'n', 'd']